In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install nltk
#nltk adalah library python yang digunakan untuk memproses nlp
#fiturnya berupa tokenization, stemming

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df_credits = pd.read_csv('/kaggle/input/netflix-tv-shows-and-movies/credits.csv')
# df_credits

In [ ]:
import pandas as pd

In [ ]:
df_titles = pd.read_csv('/content/drive/MyDrive/netflix/titles.csv') # dari google drive
# df_titles = pd.read_csv('https://drive.google.com/file/d/1gjAJykbm_3ZXeAHH213j-KNlXr3NAVgz/view?usp=sharing')  # dari link share google drive
df_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,NaN,100,"['romance', 'drama']",['NG'],NaN,tt13857480,6.8,45.0,1.466,NaN
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021,NaN,134,['drama'],[],NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,['comedy'],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021,PG-13,37,[],['US'],NaN,NaN,NaN,NaN,1.296,10.000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# PREPROCESSING

In [ ]:
import string
import pickle
import matplotlib.pyplot as plt
#pickle adalah untuk menyimpan model

from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer # fungsi yang digunakan untuk mengubah teks menjadi representasi vektor dengan bobot yang lebih akurat daripada CountVectorizer.
from sklearn.feature_extraction.text import CountVectorizer #menghitung jumlah kemunculan setiap kata pada dokumen dan menghasilkan matriks dokumen-kata
from sklearn.feature_selection import chi2

## 1. Judul dan Deskripsi

### 1.1 Drop NA

In [ ]:
df_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

In [ ]:
# drop na
df_titles = df_titles.dropna()
df_titles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1667 entries, 5 to 5831
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1667 non-null   object 
 1   title                 1667 non-null   object 
 2   type                  1667 non-null   object 
 3   description           1667 non-null   object 
 4   release_year          1667 non-null   int64  
 5   age_certification     1667 non-null   object 
 6   runtime               1667 non-null   int64  
 7   genres                1667 non-null   object 
 8   production_countries  1667 non-null   object 
 9   seasons               1667 non-null   float64
 10  imdb_id               1667 non-null   object 
 11  imdb_score            1667 non-null   float64
 12  imdb_votes            1667 non-null   float64
 13  tmdb_popularity       1667 non-null   float64
 14  tmdb_score            1667 non-null   float64
dtypes: float64(5), int64(

In [ ]:
df_titles['age_certification'].value_counts()

TV-MA    822
TV-14    417
TV-PG    161
TV-Y7    106
TV-Y      87
TV-G      74
Name: age_certification, dtype: int64

### 1.2 Bersihkan Data Teks

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text) -> string:
#     ubah ke huruf kecil
    text = str(text).lower()
#     hilangkan simbol
    text = "".join([char for char in text if (char not in string.punctuation and not char.isdigit())])
#     hilangkan stopword
    stop_words = stopwords.words('english')
    list_word = [word.replace(' ','') for word in text.split(' ') if word.replace(' ','') not in stop_words]
#     stem
    porter = PorterStemmer()
    list_word = [porter.stem(word) for word in list_word]
    return ' '.join(list_word)


In [ ]:
df = df_titles[['title', 'description']]
title = df_titles['title'].apply(clean_text)
desc = df_titles['description'].apply(clean_text)
df_title_desc_clean = pd.DataFrame({"title": df_titles["title"], "desc": df_titles["description"], "title_clean": title, "desc_clean": desc})
df_title_desc_clean.to_csv("title_desc_clean.csv")
#kolom yang ingin dibersihkan

In [ ]:
df_title_desc_clean = pd.DataFrame({"title": title, "desc": desc})
df_title_desc_clean.to_csv("title_desc_clean.csv")

### 1.3 Encode Teks

#### 1.3.1 TF-IDF Vectorizer

In [ ]:
vectorizer_title = TfidfVectorizer(max_features=500)#diambil 500 karena jumlah fitur yang dihasilkan tidak banyak dan mempercepat proses pelatihan model
X_title = vectorizer_title.fit_transform(title)

# save vectorizer
with open("/vectorizer_title.pkl", 'wb') as handle:
    pickle.dump(vectorizer_title, handle)

print(X_title.shape)

(1667, 500)


In [ ]:
vectorizer_desc = TfidfVectorizer(max_features=500)
X_desc = vectorizer_desc.fit_transform(desc)

# save vectorizer
# kaggle_output_dir = '/kaggle/working/'
with open("/vectorizer_desc.pkl", 'wb') as handle:
    pickle.dump(vectorizer_desc, handle)

print(X_desc.shape)

(1667, 500)


In [ ]:
vectorizer_title.get_feature_names_out()

array(['academi', 'adventur', 'age', 'al', 'alia', 'alien', 'almost',
       'alon', 'amaz', 'america', 'american', 'anatomi', 'angri', 'anim',
       'anitta', 'anoth', 'apach', 'arcadia', 'art', 'ash', 'asia',
       'attack', 'avatar', 'away', 'babi', 'babysitt', 'back', 'bad',
       'bake', 'baki', 'bangkok', 'bank', 'barbi', 'basketbal', 'battl',
       'bear', 'beast', 'beat', 'beauti', 'bebop', 'becom', 'behind',
       'berlin', 'best', 'better', 'beyond', 'big', 'bill', 'bird',
       'black', 'blind', 'block', 'blood', 'blow', 'blown', 'blue',
       'borgen', 'boss', 'boy', 'brazil', 'break', 'british', 'bu',
       'buddi', 'busi', 'call', 'car', 'case', 'cat', 'challeng',
       'champion', 'chanc', 'chef', 'children', 'chosen', 'christma',
       'circl', 'citi', 'club', 'code', 'comedi', 'convers', 'cook',
       'cop', 'countri', 'cowboy', 'creatur', 'crew', 'crime', 'crimin',
       'crook', 'crow', 'crown', 'crystal', 'cuervo', 'curiou', 'curs',
       'cut', 'dad', 

In [ ]:
vectorizer_desc.get_feature_names_out()

array(['abil', 'academi', 'accid', 'accident', 'accus', 'across', 'actor',
       'adventur', 'age', 'agent', 'ago', 'air', 'alien', 'along', 'also',
       'alway', 'america', 'american', 'ancient', 'angel', 'anim',
       'anoth', 'answer', 'anyth', 'apart', 'appear', 'around', 'art',
       'artist', 'attempt', 'away', 'babi', 'back', 'bad', 'baker',
       'band', 'base', 'battl', 'beauti', 'becom', 'begin', 'behind',
       'belov', 'best', 'big', 'bodi', 'bond', 'book', 'boss', 'boy',
       'bring', 'broadcast', 'brother', 'build', 'busi', 'call', 'car',
       'care', 'career', 'case', 'celebr', 'center', 'centuri',
       'challeng', 'chanc', 'chang', 'charact', 'charm', 'chef', 'chen',
       'child', 'childhood', 'children', 'citi', 'close', 'club',
       'collect', 'colleg', 'come', 'comedi', 'comedian', 'commun',
       'compani', 'compet', 'competit', 'complet', 'complic', 'conflict',
       'connect', 'conspiraci', 'control', 'cook', 'cop', 'corrupt',
       'could', 'c

In [ ]:
df_title_vectorize = pd.DataFrame(X_title.toarray(), columns = 'title_' + vectorizer_title.get_feature_names_out())
df_desc_vectorize = pd.DataFrame(X_desc.toarray(), columns = 'desc_' + vectorizer_desc.get_feature_names_out())

X_text = pd.concat([df_title_vectorize, df_desc_vectorize], axis=1)

In [ ]:
X_text

,title_academi,title_adventur,title_age,title_al,title_alia,title_alien,title_almost,title_alon,title_amaz,title_america,...,desc_without,desc_woman,desc_women,desc_work,desc_world,desc_year,desc_yearold,desc_yet,desc_york,desc_young
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.615568,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.398714,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.442898,0.000000,0.0
1663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0
1665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.237407,0.0,0.0,0.000000,0.000000,0.0


#### 1.3.1.b Count Vectorizer

In [ ]:
vectorizer_title = CountVectorizer(max_features=500)
X_title = vectorizer_title.fit_transform(title)

# save vectorizer
# kaggle_output_dir = '/kaggle/working/'
with open("vectorizer_title.pkl", 'wb') as handle:
    pickle.dump(vectorizer_title, handle)

print(X_title.shape)

(1667, 500)


In [ ]:
vectorizer_desc = CountVectorizer(max_features=500)
X_desc = vectorizer_desc.fit_transform(desc)

# save vectorizer
# kaggle_output_dir = '/kaggle/working/'
with open("vectorizer_desc.pkl", 'wb') as handle:
    pickle.dump(vectorizer_desc, handle)

print(X_desc.shape)

(1667, 500)


In [ ]:
df_title_vectorize = pd.DataFrame(X_title.toarray(), columns = 'title_' + vectorizer_title.get_feature_names_out())
df_desc_vectorize = pd.DataFrame(X_desc.toarray(), columns = 'desc_' + vectorizer_desc.get_feature_names_out())

X_text = pd.concat([df_title_vectorize, df_desc_vectorize], axis=1)
X_text

,title_academi,title_adventur,title_age,title_al,title_alia,title_alien,title_almost,title_alon,title_amaz,title_america,...,desc_without,desc_woman,desc_women,desc_work,desc_world,desc_year,desc_yearold,desc_yet,desc_york,desc_young
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1664,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


#### 1.3.2 One-Hot Encoding

In [ ]:
X_text_biner = pd.DataFrame()
for col in X_text.columns:
    X_text_biner[col] = X_text[col].apply(lambda x: 1 if x > 0 else 0)
X_text_biner

<ipython-input-25-9decb00659bd>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_text_biner[col] = X_text[col].apply(lambda x: 1 if x > 0 else 0)
<ipython-input-25-9decb00659bd>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_text_biner[col] = X_text[col].apply(lambda x: 1 if x > 0 else 0)
<ipython-input-25-9decb00659bd>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

,title_academi,title_adventur,title_age,title_al,title_alia,title_alien,title_almost,title_alon,title_amaz,title_america,...,desc_without,desc_woman,desc_women,desc_work,desc_world,desc_year,desc_yearold,desc_yet,desc_york,desc_young
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1663,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1664,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1665,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


## 2. Genre

In [ ]:
list_genre = []
for genres in df_titles['genres']:
    list_genre_temp = eval(genres)
    for g in list_genre_temp:
        if g not in list_genre:
            list_genre.append(g)
list_genre

['comedy',
 'european',
 'scifi',
 'action',
 'crime',
 'drama',
 'animation',
 'family',
 'fantasy',
 'music',
 'romance',
 'reality',
 'western',
 'thriller',
 'horror',
 'sport',
 'documentation',
 'war',
 'history']

In [ ]:
X_cat = pd.DataFrame()
for g in list_genre:
    X_cat['genre_' + g] = df_titles['genres'].map(lambda genres: 1 if g in eval(genres) else 0)
X_cat = X_cat.reset_index()
X_cat = X_cat.drop(columns='index')

In [ ]:
X_cat

,genre_comedy,genre_european,genre_scifi,genre_action,genre_crime,genre_drama,genre_animation,genre_family,genre_fantasy,genre_music,genre_romance,genre_reality,genre_western,genre_thriller,genre_horror,genre_sport,genre_documentation,genre_war,genre_history
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1663,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1664,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1665,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0


## 3. Type

In [ ]:
encoder_type = LabelEncoder()
X_cat['type'] = encoder_type.fit_transform(df_titles['type'])

# kaggle_output_dir = '/kaggle/working/'
with open("/encoder_type.pkl", 'wb') as handle:
    pickle.dump(encoder_type, handle)

print(X_cat.shape)

(1667, 20)


In [ ]:
X_cat.head()

,genre_comedy,genre_european,genre_scifi,genre_action,genre_crime,genre_drama,genre_animation,genre_family,genre_fantasy,genre_music,genre_romance,genre_reality,genre_western,genre_thriller,genre_horror,genre_sport,genre_documentation,genre_war,genre_history,type
0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0


## 4. Age Certification

In [ ]:
encoder_age_certif = LabelEncoder()
y = encoder_age_certif.fit_transform(df_titles['age_certification'])

with open("/encoder_age_certif.pkl", 'wb') as handle:
    pickle.dump(encoder_age_certif, handle)

print(y.shape)

(1667,)


In [ ]:
encoder_age_certif.classes_

array(['TV-14', 'TV-G', 'TV-MA', 'TV-PG', 'TV-Y', 'TV-Y7'], dtype=object)

In [ ]:
encoder_age_certif.transform(['TV-14', 'TV-G', 'TV-MA', 'TV-PG', 'TV-Y', 'TV-Y7'])

array([0, 1, 2, 3, 4, 5])

# BUILD MODEL

In [ ]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## CASE 1: Categorical Naive-Bayes







### 1.1 Split Data

In [ ]:
X = pd.concat([X_cat, X_text_biner], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=124, test_size=0.25, stratify=y)

### 1.2 Feature Selection

In [ ]:
chi2_res = chi2(X_train, y_train)
list_X_idx_col_not_sig = np.where(chi2_res[1] >= 0.05)

list_X_col_name_not_sig = X_train.columns[list_X_idx_col_not_sig]
X_train = X_train.drop(columns=list_X_col_name_not_sig)
X_test = X_test.drop(columns=list_X_col_name_not_sig)

### 1.3 Train dan Evaluasi Model

In [ ]:
clf = CategoricalNB(min_categories=2)
clf.fit(X_train, y_train)#overfitting

# kaggle_output_dir = '/kaggle/working/'
with open("/clf.pkl", 'wb') as handle:
    pickle.dump(clf, handle)

print('Train:')
print(classification_report(y_train, clf.predict(X_train)))

print('Test:')
print(classification_report(y_test, clf.predict(X_test)))

Train:
              precision    recall  f1-score   support

           0       0.55      0.41      0.47       313
           1       0.83      0.36      0.50        56
           2       0.71      0.87      0.78       616
           3       0.53      0.35      0.42       121
           4       0.73      0.74      0.73        65
           5       0.62      0.80      0.70        79

    accuracy                           0.67      1250
   macro avg       0.66      0.59      0.60      1250
weighted avg       0.65      0.67      0.65      1250

Test:
              precision    recall  f1-score   support

           0       0.40      0.31      0.35       104
           1       0.75      0.33      0.46        18
           2       0.66      0.79      0.72       206
           3       0.36      0.25      0.29        40
           4       0.89      0.77      0.83        22
           5       0.57      0.74      0.65        27

    accuracy                           0.59       417
   macro a

### 1.4 Save Prediksi Model

In [ ]:
X_sig_feature = X.drop(columns=list_X_col_name_not_sig)
y_pred_1 = clf.predict(X_sig_feature)
df_case_1 = pd.concat([df.reset_index(), X_sig_feature] , axis=1)
df_case_1['age_certification'] = y
df_case_1['predict_age_certification'] = y_pred_1
df_case_1 = df_case_1.drop(columns='index')
df_case_1.to_csv('df_case_1.csv', index=False)

df_case_1

,title,description,genre_comedy,genre_european,genre_scifi,genre_action,genre_crime,genre_drama,genre_animation,genre_family,...,desc_two,desc_use,desc_warrior,desc_wild,desc_win,desc_world,desc_year,desc_yearold,age_certification,predict_age_certification
0,Monty Python's Flying Circus,A British sketch comedy series with the shows ...,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,Monty Python's Fliegender Zirkus,Monty Python's Fliegender Zirkus consisted of ...,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2,0
2,Seinfeld,A stand-up comedian and his three offbeat frie...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2
3,Knight Rider,"Michael Long, an undercover police officer, is...",0,0,1,1,1,1,0,0,...,1,0,0,0,0,0,0,0,3,2
4,Thomas & Friends,Thomas & Friends is a British children's telev...,1,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,The Wedding Coach,Former bride and forever comedian Jamie Lee sh...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
1663,Only Jokes Allowed,Six of South Africa's top comedians take cente...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2
1664,We Are: The Brooklyn Saints,A Brooklyn youth football program and its self...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1665,Alma Matters: Inside the IIT Dream,"In a ""nation of middle-class"" the IIT dream in...",0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,2,2
